<a href="https://colab.research.google.com/github/malynkovsky/dimploma_bachelor/blob/master/Dimplom_v1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import xlrd
from queue import PriorityQueue
from math import inf
import copy
import random
import timeit
import math

In [ ]:
file_loc="/content/drive/My Drive/Eng/votes_of_9th_parlament(01-11-2019_31-12-2019).xlsx"
file_loc="/content/votes_.xlsx"
wkb=xlrd.open_workbook(file_loc)
sheet=wkb.sheet_by_index(0)
print((sheet.nrows-1,sheet.ncols-1))
_matrix=np.zeros((sheet.nrows-1,sheet.ncols-1))
for col in range(1,sheet.ncols):
    for row in range(1,sheet.nrows):
      _matrix[row-1,col-1] = sheet.cell_value(row,col)
print(_matrix.shape)

(773, 171)
(773, 171)


In [ ]:
test_vote_matrix = _matrix
test_dist_matrix = np.zeros((_matrix.shape[0],_matrix.shape[0]))
for i in range(test_dist_matrix.shape[0]):
  for j in range(test_dist_matrix.shape[0]):
    test_dist_matrix[i,j]=np.linalg.norm(test_vote_matrix[i] - test_vote_matrix[j])
    if (i==j):
      test_dist_matrix[i,j] = 2*_matrix.shape[0]
print(test_vote_matrix.shape)
print(test_dist_matrix.shape)
test_adj_matrix = np.zeros(test_dist_matrix.shape)
for i in range(_matrix.shape[0]):
  rrrrrr = test_dist_matrix[i]
  polk= pd.Series(-rrrrrr)
  cdel = polk.nlargest(4, keep='all')
  test_adj_matrix[i,cdel.index]=1
for i in range(test_adj_matrix.shape[0]):
  for j in range(i,test_adj_matrix.shape[1]):
    if (test_adj_matrix[i][j] == 1) or (test_adj_matrix[j][i] == 1):
      test_adj_matrix[i][j] = 1
      test_adj_matrix[j][i] = 1
    if (i == j):
      test_adj_matrix[i][i] = 0
rand_start = np.zeros(test_adj_matrix.shape)
for i in range(test_adj_matrix.shape[0]):
  for j in range(test_adj_matrix.shape[1]):
    rand_start[i,j] = np.random.randint(1,6)

(773, 171)
(773, 773)


In [ ]:
def show_graph_with_labels(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    weights = rand_start[np.where(adjacency_matrix == 1)]
    edges = zip(rows.tolist(), cols.tolist(),weights.tolist() )
    gr=nx.Graph()
    gr.add_weighted_edges_from(edges)
    graph_test = gr
    return gr

In [ ]:
def draw_graph_with_labels(gr):
    plt.figure(figsize=(20,20))
    pos = nx.layout.fruchterman_reingold_layout(gr)
    nx.draw_networkx_edges(gr, pos, edge_color='green', width = 0.5 )
    nx.draw_networkx_edge_labels(gr,pos,edge_labels=nx.get_edge_attributes(gr,'weight'),bbox=dict(alpha=0)) #
    nx.draw_networkx_nodes(gr, pos, node_color='#65ebdd', node_size=500) 
    nx.draw_networkx_labels(gr, pos, font_size=16) # labels = labels, font_size = 16, font_color = 'black')
    plt.show()
    print(gr.nodes)
    return gr


In [ ]:
def draw_small_graph(gr):
    plt.figure(figsize=(10,10))
    pos = nx.layout.fruchterman_reingold_layout(gr)
    nx.draw_networkx_edges(gr, pos, edge_color='green', width = 0.5)
    nx.draw_networkx_nodes(gr, pos, node_color='red', node_size=4) 
    plt.show()
    #print(gr.nodes)
    return gr

In [ ]:
graph_test = show_graph_with_labels(test_adj_matrix)
graphs_list_test = list(nx.algorithms.connected_components(graph_test))
conn_idx=[]
for i in range(len(graphs_list_test)):
  for j in range(i,len(graphs_list_test)):
    min_dist=2*_matrix.shape[0]
    for m in graphs_list_test[i]:
      for c in graphs_list_test[j]:
        if (test_dist_matrix[m,c] < min_dist):
          min_dist= test_dist_matrix[m,c]
    
    for m in graphs_list_test[i]:
      for c in graphs_list_test[j]:
        if (test_dist_matrix[m,c] == min_dist):
          conn_idx.append((m,c))
if (nx.algorithms.number_connected_components(graph_test) != 1):
    for t,j in conn_idx:
          graph_test.add_edge(t, j, weight=1) 

In [ ]:
def Sorting(lst): 
    lst2 = sorted(lst, key=len,reverse=True)
    return lst2

In [ ]:
def dijkstra_TEST5(graph: 'networkx.classes.graph.Graph', start: str, end: str) -> 'List':
    def backtrace(prev, start, end):
        node = end
        path = []
        while node != start:
            path.append(node)
            node = prev[node]
        path.append(node) 
        path.reverse()
        return path

    def cost(u, v):
        return graph.get_edge_data(u,v).get('weight')

    prev = {} 
    dist = {v: inf for v in list(nx.nodes(graph))} 
    visited = set() 
    pq = PriorityQueue()  
    dist[start] = 0
    pq.put((dist[start], start))    
    while 0 != pq.qsize():
        curr_cost, curr = pq.get()
        if curr == end:
            break
        if curr in visited:
          continue
        visited.add(curr)
        for neighbor in dict(graph.adjacency()).get(curr):
          if neighbor in visited:
            continue
          path = max(dist[curr],cost(curr,neighbor))
          if path < dist[neighbor]:
            dist[neighbor] = path
            prev[neighbor] = curr
            pq.put((dist[neighbor],neighbor))
               
    return backtrace(prev, start, end), dist[end]
 

In [ ]:
start = timeit.default_timer()
temp_graph = copy.deepcopy(graph_test)
F_max = 5
num_of_groups=2
result_components=[]
for u,v,w in temp_graph.edges(data=True):
      graph_test[u][v]['weight'] = np.random.randint(1,F_max+1)
for iteration_num in range(2000):
  print("Iteration: "+str(iteration_num))
  check_comp_num = True
  while (check_comp_num):   
    a,b = np.random.choice(graph_test.nodes,size=2)
    resu_testing = dijkstra_TEST5(graph_test,a,b) #step 2, 3
    if (resu_testing[1] == F_max): #step 5
      list_of_max_weights=[]
      for u,v,w in graph_test.edges(data=True):#step 5.1  удаление ребер с максимальной частотой  
        if (w['weight'] == F_max):
          temp_graph.remove_edge(u,v)
          list_of_max_weights.append((u,v))
      curr_gr = nx.algorithms.number_connected_components(temp_graph)
      if (curr_gr == num_of_groups):
        check_comp_num = False
        graphs_list_test = list(nx.algorithms.connected_components(temp_graph))
        print(len(graphs_list_test))
        ans=[]
        for t in range(len(graphs_list_test)):
          ans.append(graphs_list_test[t])
        result_components.append(ans) 
        for i in range(len(resu_testing[0])-1): #step 4
          graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1  
        F_max += 1
      if (curr_gr > num_of_groups ):#step 5.2 - изучаем шо вышло
        check_comp_num = False
        raw_list = list(nx.algorithms.connected_components(temp_graph))
        graphs_list_test = Sorting(raw_list)
        print(len(graphs_list_test))
        ans=[]
        for j in range(num_of_groups-1):
          ans.append(graphs_list_test[j])
        last_gr=[]
        for j in range(len(graphs_list_test)-num_of_groups+1):
          for t in range(len(graphs_list_test[num_of_groups-1+j])):
            last_gr.append(list(graphs_list_test[num_of_groups-1+j])[t])
        ans.append(last_gr)
        result_components.append(ans) 
        F_max += 1
        for i in range(len(resu_testing[0])-1): #step 4
          graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1
      

      for t,j in list_of_max_weights:
        temp_graph.add_edge(t, j, weight=1)
      
    else:
      for i in range(len(resu_testing[0])-1): #step 4
        graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1

stop = timeit.default_timer()

print('Time: ', stop - start) 


Iteration: 0
7
Iteration: 1
3
Iteration: 2
2
Iteration: 3
2
Iteration: 4
2
Iteration: 5
2
Iteration: 6
2
Iteration: 7
2
Iteration: 8
2
Iteration: 9
2
Iteration: 10
2
Iteration: 11
2
Iteration: 12
2
Iteration: 13
2
Iteration: 14
2
Iteration: 15
2
Iteration: 16
2
Iteration: 17
2
Iteration: 18
2
Iteration: 19
2
Iteration: 20
2
Iteration: 21
2
Iteration: 22
2
Iteration: 23
2
Iteration: 24
2
Iteration: 25
2
Iteration: 26
2
Iteration: 27
2
Iteration: 28
2
Iteration: 29
2
Iteration: 30
2
Iteration: 31
2
Iteration: 32
2
Iteration: 33
2
Iteration: 34
2
Iteration: 35
2
Iteration: 36
2
Iteration: 37
2
Iteration: 38
2
Iteration: 39
2
Iteration: 40
2
Iteration: 41
2
Iteration: 42
2
Iteration: 43
2
Iteration: 44
2
Iteration: 45
2
Iteration: 46
2
Iteration: 47
2
Iteration: 48
2
Iteration: 49
2
Iteration: 50
2
Iteration: 51
2
Iteration: 52
2
Iteration: 53
2
Iteration: 54
2
Iteration: 55
2
Iteration: 56
2
Iteration: 57
2
Iteration: 58
2
Iteration: 59
2
Iteration: 60
2
Iteration: 61
2
Iteration: 62
2
It

In [ ]:
unique_list=[]
for t in range(len(result_components)):
  if (result_components[t]) not in unique_list:
    unique_list.append(result_components[t])
unique_dec=[]
for t in range(len(unique_list)):
  dec_count=0
  for j in range(len(result_components)):
    if (result_components[j] == unique_list[t]):
      dec_count += 1
  unique_dec.append(dec_count)
sum_part=0
for t in range(len(unique_dec)):
  sum_part -= (unique_dec[t]/2000)*math.log(unique_dec[t]/2000)
print(sum_part/math.log(2000))
print(len(unique_list)/2000)
print(num_of_groups)
print(file_loc)

0.0021716705998685317
0.002
2
/content/drive/My Drive/Eng/votes_of_9th_parlament(01-11-2019_31-12-2019).xlsx


In [ ]:
start = timeit.default_timer()
temp_graph = copy.deepcopy(graph_test)
F_max = 5
num_of_groups=3
result_components=[]
for u,v,w in temp_graph.edges(data=True):
      graph_test[u][v]['weight'] = np.random.randint(1,F_max+1)
for iteration_num in range(2000):
  print("Iteration: "+str(iteration_num))
  check_comp_num = True
  while (check_comp_num):   
    a,b = np.random.choice(graph_test.nodes,size=2)
    resu_testing = dijkstra_TEST5(graph_test,a,b) #step 2, 3
    if (resu_testing[1] == F_max): #step 5
      list_of_max_weights=[]
      for u,v,w in graph_test.edges(data=True):#step 5.1  удаление ребер с максимальной частотой  
        if (w['weight'] == F_max):
          temp_graph.remove_edge(u,v)
          list_of_max_weights.append((u,v))
      curr_gr = nx.algorithms.number_connected_components(temp_graph)
      if (curr_gr == num_of_groups):
        check_comp_num = False
        graphs_list_test = list(nx.algorithms.connected_components(temp_graph))
        print(len(graphs_list_test))
        ans=[]
        for t in range(len(graphs_list_test)):
          ans.append(graphs_list_test[t])
        result_components.append(ans) 
        for i in range(len(resu_testing[0])-1): #step 4
          graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1  
        F_max += 1
      if (curr_gr > num_of_groups ):#step 5.2 - изучаем шо вышло
        check_comp_num = False
        raw_list = list(nx.algorithms.connected_components(temp_graph))
        graphs_list_test = Sorting(raw_list)
        print(len(graphs_list_test))
        ans=[]
        for j in range(num_of_groups-1):
          ans.append(graphs_list_test[j])
        last_gr=[]
        for j in range(len(graphs_list_test)-num_of_groups+1):
          for t in range(len(graphs_list_test[num_of_groups-1+j])):
            last_gr.append(list(graphs_list_test[num_of_groups-1+j])[t])
        ans.append(last_gr)
        result_components.append(ans) 
        F_max += 1
        for i in range(len(resu_testing[0])-1): #step 4
          graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1
      

      for t,j in list_of_max_weights:
        temp_graph.add_edge(t, j, weight=1)
      
    else:
      for i in range(len(resu_testing[0])-1): #step 4
        graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1

stop = timeit.default_timer()

print('Time: ', stop - start) 

Iteration: 0
3
Iteration: 1
6
Iteration: 2
4
Iteration: 3
3
Iteration: 4
3
Iteration: 5
3
Iteration: 6
3
Iteration: 7
3
Iteration: 8
3
Iteration: 9
3
Iteration: 10
4
Iteration: 11
3
Iteration: 12
3
Iteration: 13
3
Iteration: 14
3
Iteration: 15
3
Iteration: 16
3
Iteration: 17
3
Iteration: 18
3
Iteration: 19
3
Iteration: 20
3
Iteration: 21
3
Iteration: 22
3
Iteration: 23
3
Iteration: 24
3
Iteration: 25
4
Iteration: 26
3
Iteration: 27
3
Iteration: 28
3
Iteration: 29
3
Iteration: 30
3
Iteration: 31
3
Iteration: 32
3
Iteration: 33
3
Iteration: 34
3
Iteration: 35
3
Iteration: 36
3
Iteration: 37
3
Iteration: 38
3
Iteration: 39
4
Iteration: 40
3
Iteration: 41
4
Iteration: 42
4
Iteration: 43
3
Iteration: 44
3
Iteration: 45
3
Iteration: 46
4
Iteration: 47
3
Iteration: 48
3
Iteration: 49
3
Iteration: 50
3
Iteration: 51
4
Iteration: 52
3
Iteration: 53
4
Iteration: 54
4
Iteration: 55
3
Iteration: 56
3
Iteration: 57
3
Iteration: 58
3
Iteration: 59
3
Iteration: 60
3
Iteration: 61
3
Iteration: 62
3
It

In [ ]:
unique_list=[]
for t in range(len(result_components)):
  if (result_components[t]) not in unique_list:
    unique_list.append(result_components[t])
unique_dec=[]
for t in range(len(unique_list)):
  dec_count=0
  for j in range(len(result_components)):
    if (result_components[j] == unique_list[t]):
      dec_count += 1
  unique_dec.append(dec_count)
sum_part=0
for t in range(len(unique_dec)):
  sum_part -= (unique_dec[t]/2000)*math.log(unique_dec[t]/2000)
print(sum_part/math.log(2000))
print(len(unique_list)/2000)
print(num_of_groups)
print(file_loc)

0.17026070375368382
0.005
3
/content/drive/My Drive/Eng/votes_of_9th_parlament(01-11-2019_31-12-2019).xlsx


In [ ]:
start = timeit.default_timer()
temp_graph = copy.deepcopy(graph_test)
F_max = 5
num_of_groups=4
result_components=[]
for u,v,w in temp_graph.edges(data=True):
      graph_test[u][v]['weight'] = np.random.randint(1,F_max+1)
for iteration_num in range(2000):
  print("Iteration: "+str(iteration_num))
  check_comp_num = True
  while (check_comp_num):   
    a,b = np.random.choice(graph_test.nodes,size=2)
    resu_testing = dijkstra_TEST5(graph_test,a,b) #step 2, 3
    if (resu_testing[1] == F_max): #step 5
      list_of_max_weights=[]
      for u,v,w in graph_test.edges(data=True):#step 5.1  удаление ребер с максимальной частотой  
        if (w['weight'] == F_max):
          temp_graph.remove_edge(u,v)
          list_of_max_weights.append((u,v))
      curr_gr = nx.algorithms.number_connected_components(temp_graph)
      if (curr_gr == num_of_groups):
        check_comp_num = False
        graphs_list_test = list(nx.algorithms.connected_components(temp_graph))
        print(len(graphs_list_test))
        ans=[]
        for t in range(len(graphs_list_test)):
          ans.append(graphs_list_test[t])
        result_components.append(ans) 
        for i in range(len(resu_testing[0])-1): #step 4
          graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1  
        F_max += 1
      if (curr_gr > num_of_groups ):#step 5.2 - изучаем шо вышло
        check_comp_num = False
        raw_list = list(nx.algorithms.connected_components(temp_graph))
        graphs_list_test = Sorting(raw_list)
        print(len(graphs_list_test))
        ans=[]
        for j in range(num_of_groups-1):
          ans.append(graphs_list_test[j])
        last_gr=[]
        for j in range(len(graphs_list_test)-num_of_groups+1):
          for t in range(len(graphs_list_test[num_of_groups-1+j])):
            last_gr.append(list(graphs_list_test[num_of_groups-1+j])[t])
        ans.append(last_gr)
        result_components.append(ans) 
        F_max += 1
        for i in range(len(resu_testing[0])-1): #step 4
          graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1
      

      for t,j in list_of_max_weights:
        temp_graph.add_edge(t, j, weight=1)
      
    else:
      for i in range(len(resu_testing[0])-1): #step 4
        graph_test[resu_testing[0][i]][resu_testing[0][i+1]]['weight'] +=1

stop = timeit.default_timer()

print('Time: ', stop - start) 

Iteration: 0
6
Iteration: 1
6
Iteration: 2
5
Iteration: 3
4
Iteration: 4
4
Iteration: 5
4
Iteration: 6
4
Iteration: 7
4
Iteration: 8
5
Iteration: 9
4
Iteration: 10
4
Iteration: 11
4
Iteration: 12
4
Iteration: 13
4
Iteration: 14
5
Iteration: 15
4
Iteration: 16
4
Iteration: 17
4
Iteration: 18
4
Iteration: 19
4
Iteration: 20
5
Iteration: 21
4
Iteration: 22
4
Iteration: 23
4
Iteration: 24
4
Iteration: 25
4
Iteration: 26
4
Iteration: 27
4
Iteration: 28
4
Iteration: 29
4
Iteration: 30
5
Iteration: 31
4
Iteration: 32
4
Iteration: 33
4
Iteration: 34
4
Iteration: 35
4
Iteration: 36
5
Iteration: 37
4
Iteration: 38
4
Iteration: 39
4
Iteration: 40
4
Iteration: 41
5
Iteration: 42
5
Iteration: 43
4
Iteration: 44
4
Iteration: 45
4
Iteration: 46
4
Iteration: 47
4
Iteration: 48
4
Iteration: 49
4
Iteration: 50
4
Iteration: 51
4
Iteration: 52
4
Iteration: 53
5
Iteration: 54
4
Iteration: 55
5
Iteration: 56
4
Iteration: 57
4
Iteration: 58
4
Iteration: 59
5
Iteration: 60
4
Iteration: 61
4
Iteration: 62
4
It

In [ ]:
unique_list=[]
for t in range(len(result_components)):
  if (result_components[t]) not in unique_list:
    unique_list.append(result_components[t])
unique_dec=[]
for t in range(len(unique_list)):
  dec_count=0
  for j in range(len(result_components)):
    if (result_components[j] == unique_list[t]):
      dec_count += 1
  unique_dec.append(dec_count)
sum_part=0
for t in range(len(unique_dec)):
  sum_part -= (unique_dec[t]/2000)*math.log(unique_dec[t]/2000)
print(sum_part/math.log(2000))
print(len(unique_list)/2000)
print(num_of_groups)
print(file_loc)

0.39124309400525437
0.029
4
/content/votes_.xlsx
